In [1]:
!pip install opensoundscape==0.6.2
from opensoundscape.preprocess.preprocessors import BasePreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures
from WTS_chunking import annotation_chunker_no_duplicates, extact_split, fill_no_class


Bad key "text.kerning_factor" on line 4 in
C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#COSMOS_BirdNET-Lite_Labels_100.csv
name = "automated_cosmos_tweety_to_file"
main_name ="C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/automated_cosmos_tweety_to_file.csv"#"./COSMOS_Microfaune-Filename_Labels_100.csv"#"./COSMOS_BirdNET-Lite-Filename_Labels_05Conf.csv" #'./COSMOS_BirdNET-Lite_Labels_05Conf.csv' #Tweety birdnet, #tweetyfilename
df = pd.read_csv(main_name, sep=',')
#warblr = pd.read_csv("./warblrb10k_public_metadata_2018.csv")
#no_bird = warblr[warblr["hasbird"] == 0].reset_index(drop=True)
#no_bird["OFFSET"] = float(0)
#no_bird["DURATION"] = float(10)
#no_bird["FOLDER"] = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/wav/"
#no_bird["CHANNEL"] = 0
#no_bird["CLIP LENGTH"] = float(10)
#no_bird["MANUAL ID"] = "no bird"
#no_bird["CONFIDENCE"] = 1
#no_bird["SAMPLE RATE"] = 44100
#no_bird["FILENAME"] = no_bird["itemid"]+".wav"
#no_bird["IN FILE"] = no_bird["FILENAME"]
no_bird = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/ICML_NO_BIRD_SAMPLE.csv")
df = df.append(no_bird)
no_bird = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/mixed_bird_binary_annotations_tweetynet_chunked.csv")
df = df.append(no_bird)


test_name = "./" +name + "_TESTING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TESTING.csv"
train_name ="./" +name + "_TRAINING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TRAINING.csv"
THIS_IS_NEW_DATA = True
#df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
print(df["IN FILE"])

0        XC100027 - Southern Nightingale-Wren - Microce...
1        XC100027 - Southern Nightingale-Wren - Microce...
2        XC100027 - Southern Nightingale-Wren - Microce...
3        XC100027 - Southern Nightingale-Wren - Microce...
4        XC100027 - Southern Nightingale-Wren - Microce...
                               ...                        
29240                       Zonotrichia-capensis-47546.wav
29241                       Zonotrichia-capensis-47546.wav
29242                       Zonotrichia-capensis-47546.wav
29243                       Zonotrichia-capensis-47546.wav
29244                       Zonotrichia-capensis-47546.wav
Name: IN FILE, Length: 115831, dtype: object


In [3]:
no_bird

,Unnamed: 0,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID
0,0,Aburria-aburri-257423.wav,24.633469,0,0.0,3.0,44100,bird
1,1,Aburria-aburri-257423.wav,24.633469,0,3.0,3.0,44100,bird
2,2,Aburria-aburri-257423.wav,24.633469,0,6.0,3.0,44100,bird
3,3,Aburria-aburri-257423.wav,24.633469,0,9.0,3.0,44100,bird
4,4,Aburria-aburri-257423.wav,24.633469,0,15.0,3.0,44100,bird
...,...,...,...,...,...,...,...,...
29240,29240,Zonotrichia-capensis-47546.wav,54.935510,0,30.0,3.0,44100,bird
29241,29241,Zonotrichia-capensis-47546.wav,54.935510,0,42.0,3.0,44100,bird
29242,29242,Zonotrichia-capensis-47546.wav,54.935510,0,45.0,3.0,44100,bird
29243,29243,Zonotrichia-capensis-47546.wav,54.935510,0,48.0,3.0,44100,bird


In [4]:
ef = pd.read_csv('C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/cosmos_labeled_data_files_added.csv')
ef = annotation_chunker_no_duplicates(ef, 3, True)
ef['IN FILE'] = ef['IN FILE'].apply(lambda x : x.replace("_", " "))
gtClips = np.array(ef['IN FILE'].unique())
ef.shape

(6621, 9)

In [5]:
ef["IN FILE"][0]

'XC87449 - Scaly-breasted Woodpecker - Celeus grammicus subcervinus.mp3'

In [6]:

j = 0
ff = ef#pd.DataFrame(columns=df.columns)
#for i, row in ef.iterrows():
#    if(row['IN FILE'] in gtClips):
#        # print(row['IN FILE'])
##        ff.loc[j] = row
#        df.drop(i, inplace=True)
#        j += 1
#print(df.shape)
try:
    df = df[df['CONFIDENCE'] >= 0.8]
except:
    pass

In [7]:
ff

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE
0,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,0.0,3.0,44100,Celeus grammicus,0.0
1,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,3.0,3.0,44100,Celeus grammicus,0.0
2,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,6.0,3.0,44100,no bird,0.0
3,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,9.0,3.0,44100,Celeus grammicus,0.0
4,./COSMOS,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0,12.0,3.0,44100,Celeus grammicus,0.0
...,...,...,...,...,...,...,...,...,...
6616,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,15.0,3.0,44100,Tolmomyias sulphurescens,0.0
6617,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,18.0,3.0,44100,no bird,0.0
6618,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,21.0,3.0,44100,Tolmomyias sulphurescens,0.0
6619,./COSMOS,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,0,24.0,3.0,44100,Tolmomyias sulphurescens,0.0


In [8]:
len(df['IN FILE'].unique())

5126

In [9]:
df['IN FILE'] = ['./Cosmos_data/Training/' + f for f in df['IN FILE']]
ff['IN FILE CURRENT'] = ['./Cosmos_data/Training_Xeno_Canto_2022/' + f for f in ff['IN FILE']]
ff['IN FILE'] = ['./Cosmos_data/Testing_Xeno_Canto_2022/' + f for f in ff['IN FILE']]


In [10]:
ef = ff.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
ef['IN FILE'] = [f.replace(".mp3", ".wav") for f in ef['IN FILE']]
import os
from pydub import AudioSegment

path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/"

#Change working directory
os.chdir(path)

audio_files = os.listdir()

# You dont need the number of files in the folder, just iterate over them directly using:
for file in audio_files:
    #spliting the file into the name and the extension
    name, ext = os.path.splitext(file)
    if ext == ".mp3":
        try:
           mp3_sound = AudioSegment.from_mp3(file)
           #rename them using the old name + ".wav"
           mp3_sound.export("{0}.wav".format(name), format="wav")
        except:
            continue
ef

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE,IN FILE CURRENT
3416,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC100864...,9.39575,0,6.0,3.0,44100,Zonotrichia capensis,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10086...
3308,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101469...,137.41000,0,39.0,3.0,44100,Turdus leucomelas,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10146...
3427,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101564...,63.97150,0,30.0,3.0,44100,Zonotrichia capensis,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10156...
4033,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101694...,23.05110,0,3.0,3.0,44100,Myioborus miniatus,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10169...
2684,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC101816...,34.43860,0,27.0,3.0,44100,Turdus leucomelas,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC10181...
...,...,...,...,...,...,...,...,...,...,...
6017,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC96443 ...,32.29380,0,15.0,3.0,44100,bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC96443...
4634,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC96647 ...,108.87900,0,36.0,3.0,48000,no bird,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC96647...
1363,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC97532 ...,20.33650,0,15.0,3.0,44100,No class of interest,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC97532...
4572,./COSMOS,./Cosmos_data/Testing_Xeno_Canto_2022/XC97542 ...,4.40037,0,0.0,3.0,44100,Myioborus miniatus,0.0,./Cosmos_data/Training_Xeno_Canto_2022/XC97542...


In [11]:
#ef["IN FILE"][2]

In [12]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    try:
        if(THIS_IS_NEW_DATA):
            shutil.copyfile(og_file_path, new_file_path)
    except:
        return "DELETE"
    return new_file_path
#Move_Files('./Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile('Cosmos_data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3', './Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile("/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3", '/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
ef["IN FILE"] = ef["IN FILE"].apply(Move_Files)


In [13]:
ef = ef[ef["IN FILE"] != "DELETE"]

In [14]:
ef["IN FILE"].iloc[0]

'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.wav'

In [15]:
"C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.mp3"


'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.mp3'

In [16]:
'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.wav'

'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC104217 - Slate-throated Whitestart - Myioborus miniatus.wav'

In [17]:
from opensoundscape.audio import Audio
import os
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0
#ef = ef.reset_index()
print(ef)
for i, row in ef.iterrows():
    try:
        rate, data = wavfile.read(row['IN FILE'])
        reduced_noise = nr.reduce_noise(y=data, sr=rate)
        wavfile.write(row['IN FILE'])
    except Exception as e:
        print(e)
        break
    
    
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception as e:
        print(e)
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('/')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix=file_name[-1],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=(os.path.exists(clip_dir + file_name[-1]))
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    print(i, "out of", ef.shape[0])

print(ef_split_save)
ef_split_save.to_csv(test_name)

        FOLDER                                            IN FILE  \
3416  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3308  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3427  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4033  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
2684  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
...        ...                                                ...   
6017  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4634  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
1363  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
4572  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   
3341  ./COSMOS  C:/Users/siloux/Desktop/E4E/passive-acoustic-b...   

      CLIP LENGTH  CHANNEL  OFFSET  DURATION  SAMPLE RATE  \
3416      9.39575        0     6.0       3.0        44100   
3308    137.41000        0    39.0       3.0     

In [18]:
df

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100027 - Southern Nig...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,./Cosmos_data/Training/XC100068 - Yellow-olive...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30934,30934,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31336.0,24000.0,0.0,217.078,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30935,30935,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31337.0,24000.0,0.0,217.080,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30936,30936,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31338.0,24000.0,0.0,217.079,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav
30937,30937,0.000000,27.559200,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,./Cosmos_data/Training/Zimmerius-gracilipes-25...,0,27.559200,44100,other,1.000000,NaN,31339.0,24000.0,0.0,217.081,alarce,10/18/2021,18:32:32,DELETE,Zimmerius-gracilipes-258669.wav


In [19]:
no_bird["FOLDER"] = ""

In [20]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    #print(new_file_path)
    name, ext = os.path.splitext(og_file_path)
    if ext == ".mp3":
        try:
           mp3_sound = AudioSegment.from_mp3(og_file_path)
           #rename them using the old name + ".wav"
           mp3_sound.export("{0}.wav".format(name), format="wav")
        except:
            return "DELETE"
    
    if(not os.path.exists(new_file_path)):
        try:
            shutil.copyfile(og_file_path, new_file_path)
        except:
            try:
                mp3_sound = AudioSegment.from_mp3(og_file_path.replace(".wav", ".mp3"))
                #rename them using the old name + ".wav"
                mp3_sound.export("{0}.wav".format(name), format="wav")
                shutil.copyfile(og_file_path, new_file_path)
            except:
                return "DELETE" 
    return new_file_path

def Move_Files_Custom(filename):
    file = filename.split("/")
    og_file_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/" + file[-1]
    new_file_path = filename
    try:
        if(not os.path.exists(new_file_path)):
            shutil.copyfile(og_file_path, new_file_path)
    except Exception as e:
        print(e)
        return "DELETE"
    return new_file_path


#df = pd.read_csv(main_name, sep=',')
df['IN FILE'] = [f.split('/')[-1].replace(".mp3", ".wav") for f in df['IN FILE']]
df['IN FILE'] = ['./Cosmos_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
df["IN FILE"] = df["IN FILE"].apply(Move_Files)
df = df[df["IN FILE"] != "DELETE"]
#warblr = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/warblrb10k_public_metadata_2018.csv")
#no_bird = warblr[warblr["hasbird"] == 0].reset_index(drop=True)
#no_bird["OFFSET"] = float(0)
#no_bird["DURATION"] = float(10)
#no_bird["FOLDER"] = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/"
#no_bird["CHANNEL"] = 0
#no_bird["CLIP LENGTH"] = float(10)
#no_bird["MANUAL ID"] = "no bird"
#no_bird["CONFIDENCE"] = 1
#no_bird["SAMPLE RATE"] = 44100
#no_bird["FILENAME"] = no_bird["itemid"]+".wav"
#no_bird["IN FILE"] = 'Training_Xeno_Canto_2022/' + no_bird["FILENAME"]
(no_bird["FOLDER"] + no_bird["IN FILE"]).apply(Move_Files_Custom)
df = df.append(no_bird)
df

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Accipiter-poliogaster-121768.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-47553.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Automolus-rufipileatus-76087.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Chaetura-viridipennis-91306.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Chaetura-viridipennis-91306.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Chaetura-viridipennis-91306.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Chaetura-viridipennis-91306.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Chaetura-viridipennis-91306.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/Mult

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Coccycua-minuta-36510.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Coccycua-minuta-36510.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Coccycua-minuta-36510.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Coccycua-minuta-36510.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Coccycua-minuta-36510.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/C

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Crypturellus-undulatus-99817.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Crypturellus-undulatus-99817.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Crypturellus-undulatus-99817.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Crypturellus-undulatus-99817.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Crypturellus-undulatus-99817.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape

...

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Troglodytes-aedon-231541.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Troglodytes-aedon-231541.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Troglodytes-aedon-231541.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Troglodytes-aedon-231541.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Troglodytes-aedon-231541.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifi

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Turdus-ignobilis-446770.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mi

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tyrannulus-elatus-494013.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tyrannulus-elatus-494013.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tyrannulus-elatus-8377.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tyrannulus-elatus-8377.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Tyrannulus-elatus-8377.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mix

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-105264.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-105264.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-105264.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-105264.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Mixed_Bird/Zenaida-auriculata-105264.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Cla

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29240,29240,30.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29241,29241,42.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29242,29242,45.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29243,29243,48.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29240,29240,30.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29241,29241,42.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29242,29242,45.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29243,29243,48.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df

,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH,Unnamed: 0.1,MAX FREQ,MIN FREQ,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FILE_PATH,FILENAME
10,10,24.906977,1.093023,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.823617,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,30.232558,0.697674,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.812980,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,50.627907,1.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,0.955846,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,60.488372,1.511628,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,81.528163,44100,Microcerculus marginatus,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,0.860465,0.465116,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,0,55.092245,44100,Tolmomyias sulphurescens,1.000000,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29240,29240,30.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29241,29241,42.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29242,29242,45.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29243,29243,48.000000,3.000000,,Zonotrichia-capensis-47546.wav,0,54.935510,44100,bird,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def fix_folder_path(filepath):
    folder = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/"
    filename = filepath.split("/")[-1]#.replace(".WAV.", ".")
    return folder + filename
df["IN FILE"] = df["IN FILE"].apply(fix_folder_path)
df["IN FILE"][0]

0    C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...
0    C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...
Name: IN FILE, dtype: object

In [24]:
if (False):
    from opensoundscape.audio import Audio
    clip_duration = 3
    clip_overlap = 0
    final_clip = None
    clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Cosmos_Data/Training_Cosmos'
    # classes = labels
    min_label_overlap = 0.1

    df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

    cnt = 0
    df = df.reset_index()
    for i, row in df.iterrows():
        try:
            audio = Audio.from_file(row['IN FILE'])
        except Exception:
            print(row['IN FILE'] + ' not found')
            continue
        file_name = row['IN FILE'].split('/')
        # print(file_name)
        clip_df = pd.DataFrame()
        offset = row["OFFSET"]
        duration = row["DURATION"]
        new_file = clip_dir + file_name[-1] + str(offset) + "s_" + str(offset+duration)+"s.wav"
        if (not os.path.exists(new_file)):
            clip_df = audio.split_and_save(
                clip_dir,
                prefix=file_name[-1],
                clip_duration=clip_duration,
                clip_overlap=clip_overlap,
                final_clip='remainder',
                dry_run=(os.path.exists(clip_dir + file_name[-1]))
            )
        else:
            tmp_df = pd.DataFrame()
            offset = row["OFFSET"]
            duration = row["DURATION"]
            tmp_df['file'] = new_file
            tmp_df['start_time'] = offset
            tmp_df['end_time'] = offset + duration
            if (clip_df.empty):
                clip_df = tmp_df
            else:
                clip_df = clip_df.append(tmp_df)

        clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
        clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
        clip_df.reset_index(inplace=True)
        df_split_save = df_split_save.append(clip_df, ignore_index=True)
        print(i, "out of", df.shape[0])

    print(df_split_save)


In [24]:
!pip install pydub
from opensoundscape.audio import Audio
from pydub import AudioSegment
#AudioSegment.converter = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
import os


clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/'
# classes = labels
min_label_overlap = 0.1

df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

def convert_df_to_ops_df(row):
    offset = row["OFFSET"]
    duration = row["DURATION"]
    in_file_curr = row['IN FILE']
    file_name = in_file_curr.split('/')
    new_file = clip_dir + file_name[-1] + "_" + str(offset) + "s_" + str(offset+duration)+"s.wav"
    row['IN FILE'] = new_file
    
    if(not os.path.exists(new_file)):
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        try:
            newAudio = AudioSegment.from_mp3(in_file_curr)
            newAudio = newAudio[t1:t2]
            new_file_object = newAudio.export(new_file, format="mp3")
            new_file_object.close()
        except Exception as e:
            print(e)
            row['IN FILE'] = "TO DELETE"
        #newAudio.close()
    
    return row
    

df_refractor = df.apply(convert_df_to_ops_df, axis=1)
df_refractor = df_refractor[df_refractor["IN FILE"] != "TO DELETE"]
df_refractor

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC148259 - Yellow-olive Flatbill - Tolmomyias sulphurescens confusus (based on locality and other XC cuts).wav_38.06976744186046s_38.76744186046512s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC148259 - Yellow-olive Flatbill - Tolmomyias sulphurescens confusus (based on locality and other XC cuts).wav_38.97674418604651s_39.55813953488372s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC211446 - Southern Nightingale-Wren - Microcerculus marginatus marginatus (ssp assumed from location).wav_0.5581395348837209s_5.116279069767442s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC212058 - Pale-breasted Thrush - Turdus leucomelas albiventer (ssp assumed from location).wav_105.27906976744184s_107.99999999999999s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC212058 - Pale-breasted Thrush - Turdus leucomelas albiventer (ssp assumed from location).wav_111.65116279069768s_113.34883720930235s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC212058 - Pale-breasted Thrush - Turdus leucomelas albiventer (ssp assumed from location).wav_113.34883720930232s_115.88372093023254s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/Open

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC269090 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_122.48837209302326s_125.81395348837209s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC269094 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_2.558139534883721s_5.5348837209302335s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC269094 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_5.558139534883721s_11.558139534883722s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/pas

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC283321 - Yellow-olive Flatbill - Tolmomyias sulphurescens pallescens (ssp assumed from location).wav_0.8604651162790697s_1.3953488372093024s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC283321 - Yellow-olive Flatbill - Tolmomyias sulphurescens pallescens (ssp assumed from location).wav_7.511627906976744s_8.953488372093023s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC283321 - Yellow-olive Flatbill - Tolmomyias sulphurescens pallescens (ssp assumed from location).wav_40.76744186046512s_41.46511627906977s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoust

[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC305391 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_1.0465116279069768s_1.9302325581395348s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC305391 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_12.27906976744186s_12.883720930232556s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/XC305391 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides (ssp assumed from location).wav_31.13953488372093s_63.13953488372093s.wav'
[Errno 2] No such file or directory: 'C:/Users/siloux/Desktop/E4E/pass

...

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Brotogeris-cyanoptera-270616.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Brotogeris-cyanoptera-270616.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Brotogeris-cyanoptera-270616.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Brotogeris-cyanoptera-270616.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Buteo-platypte

[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Calliphlox-amethystina-468193.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Calliphlox-amethystina-468193.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Calliphlox-amethystina-468193.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Calliphlox-amethystina-468193.wav'
[Errno 2] No such file or directory: 'C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/Calliphlox

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-b629a9972d62>", line 42, in <module>
    df_refractor = df.apply(convert_df_to_ops_df, axis=1)
  File "C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\frame.py", line 8740, in apply
    return op.apply()
  File "C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py", line 688, in apply
    return self.apply_standard()
  File "C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py", line 812, in apply_standard
    results, res_index = self.apply_series_generator()
  File "C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\pandas\core\apply.py", line 828, in apply_series_generator
    results[i] = self.f(v)
  File "<ipython-input-25-b629a9972d62>", line 32, in convert_df_to_ops_df


TypeError: object of type 'NoneType' has no len()

In [ ]:
df_refractor

In [ ]:
df_split_save = df_refractor[["IN FILE", "OFFSET", "DURATION", "SAMPLE RATE", "MANUAL ID"]]
df_split_save["end_time"] = df_split_save["OFFSET"] + df_split_save["DURATION"]
df_split_save = df_split_save.rename(columns={"IN FILE": "file", "OFFSET": "start_time"})
df_split_save = df_split_save[['file', 'start_time', 'end_time', 'SAMPLE RATE', 'MANUAL ID']]

In [ ]:
df_split_save

In [ ]:
train_name

In [ ]:
df_split_save.to_csv(train_name)
df_split_save

In [ ]:
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)
df["merge file"] = df["IN FILE"].apply(lambda x: x.split("/")[-1])
#df_split_save['CONFIDENCE'] = df_split_save['CONFIDENCE'].fillna(-1)
#for i in range(df.shape[0]):
##    if (i % 100 == 0):
#        print(i)
#    row = df.iloc[i]
#    rows_to_change = (df_split_save["file"].str.contains(row["merge file"])) & (df_split_save["start_time"] == (row["Start (s)"])) & (df_split_save["end_time"] == (row["End (s)"]))
#    df_split_save.loc[rows_to_change, "CONFIDENCE"]= row["Confidence"]
df_split_save

In [ ]:
df_split_save.drop_duplicates()

In [ ]:
import os
from os import listdir
from os.path import isfile, join

test_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos/"
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]
train_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/"
train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
overlapping_files = []
for item in train_files:
    if (item in test_files):
        path = train_path + "/" + item
        os.remove(path)
        overlapping_files.append(path)

df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))
df_split_save2

In [ ]:
test_path2 = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Testing_Cosmos/"
test_files2 = [f for f in listdir(test_path2) if isfile(join(test_path2, f))]
train_path2 =  "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/temp_clips/Training_Cosmos/"
train_files2 = [f for f in listdir(train_path2) if isfile(join(train_path2, f))]
overlapping_files2 = []
for item in train_files2:
    if (item in test_files2):
        overlapping_files2.append(item)
overlapping_files2

In [ ]:
df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))

In [ ]:
#df_split_save2.loc[df_split_save2["CONFIDENCE"] == -1, "CONFIDENCE"] = 0
#df_split_save2 = df_split_save2.loc[df_split_save2["CONFIDENCE"] > 0.5]
df_split_save2.to_csv(train_name) 

In [ ]:
df_split_save2

In [ ]:
np.unique(df_split_save2["MANUAL ID"])